In [ ]:
import os
from os import path
from config import DATA_PATH
import pandas as pd
import numpy as np
import spatialdata as sd
import os.path as osp
from skimage.measure import regionprops

from data import patch_john, patch_harry

In [ ]:
sdata = sd.read_zarr(osp.join(DATA_PATH, "UC6_I.zarr/UC6_I.zarr"))


/home/b-evelyntong/hl/ac109_project/raw/UC6_I.zarr/UC6_I.zarr


/home/b-evelyntong/miniconda3/envs/cs109b/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [ ]:
image = np.transpose(sdata.images['HE_original'].to_numpy(), (1, 2, 0))
mask = sdata.images["HE_nuc_original"].to_numpy()[0, :, :]

cell_ids = sdata.tables["anucleus"].obs['cell_id'].values
gene_expression_df = sdata.tables["anucleus"].to_df()

In [23]:
# check the current parent directory
print("Current working directory:", os.getcwd())

Current working directory: /home/b-evelyntong/hl/gut-instinct/notebooks


In [28]:
# let's first try out processing data
# Get cell coordinates
regions = [region for region in regionprops(mask) if region.label in cell_ids]
cell_coords = {region.label: (int(region.centroid[1]), int(region.centroid[0])) for region in regions}

In [ ]:
# caution, this block will run for ever :)
output_path = "../data/processed/patches_and_expressions.h5"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
patch_john.generate_and_save_patches(image, sdata, cell_coords, gene_expression_df,
                                     output_file=output_path)

patches, expressions = patch_john.load_patches_and_expressions(h5_file=output_path)

KeyboardInterrupt: 

In [ ]:
dataset_patch_train, dataset_patch_val, dataset_patch_test = patch_harry.get_patches(sdata)
dataset_expression_train = patch_harry.get_expression(sdata, dataset_patch_train)
dataset_expression_val = patch_harry.get_expression(sdata, dataset_patch_val)
dataset_expression_test = patch_harry.get_expression(sdata, dataset_patch_test)